<a href="https://colab.research.google.com/github/zakonreal/ds_homework/blob/main/HW_DL_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import datetime
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
data_dir = 'drive/My Drive/'
train_lang = 'en'

In [4]:
import torch
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence

class DatasetSeq(Dataset):
    def __init__(self, data_dir, train_lang='en'):
	#open file
        with open(data_dir + train_lang + '.train', 'r') as f:
            train = f.read().split('\n\n')

        # delete extra tag markup
        train = [x for x in train if not '_ ' in x]
	    #init vocabs of tokens for encoding {<str> token: <int> id}
        self.target_vocab = {} # {p: 1, a: 2, r: 3, pu: 4}
        self.word_vocab = {} # {cat: 1, sat: 2, on: 3, mat: 4, '.': 5}
        self.char_vocab = {} # {c: 1, a: 2, t: 3, ' ': 4, s: 5}
	    
        # Cat sat on mat. -> [1, 2, 3, 4, 5]
        # p    a  r  p pu -> [1, 2, 3, 1, 4]
        # chars  -> [1, 2, 3, 4, 5, 2, 3, 4]

	    #init encoded sequences lists (processed data)
        self.encoded_sequences = []
        self.encoded_targets = []
        self.encoded_char_sequences = []
        # n=1 because first value is padding
        n_word = 1
        n_target = 1
        n_char = 1
        for line in train:
            sequence = []
            target = []
            chars = []
            for item in line.split('\n'):
                if item != '':
                    word, label = item.split(' ')

                    if self.word_vocab.get(word) is None:
                        self.word_vocab[word] = n_word
                        n_word += 1
                    if self.target_vocab.get(label) is None:
                        self.target_vocab[label] = n_target
                        n_target += 1
                    for char in word:
                        if self.char_vocab.get(char) is None:
                            self.char_vocab[char] = n_char
                            n_char += 1
                    sequence.append(self.word_vocab[word])
                    target.append(self.target_vocab[label])
                    chars.append([self.char_vocab[char] for char in word])
            self.encoded_sequences.append(sequence)
            self.encoded_targets.append(target)
            self.encoded_char_sequences.append(chars)

    def __len__(self):
        return len(self.encoded_sequences)

    def __getitem__(self, index):
        return {
            'data': self.encoded_sequences[index], # [1, 2, 3, 4, 6] len=5
            'char': self.encoded_char_sequences[index],# [[1,2,3], [4,5], [1,2], [2,6,5,4], []] len=5
            'target': self.encoded_targets[index], #  (1)
        }

In [5]:
dataset = DatasetSeq(data_dir)

In [6]:
#padding
# seq1 = [1, 2, 3, 4]
# seq2 = [9, 7, 6, 4, 3, 7, 5]
# pad seq1 equal seq2
# seq1 = [1, 2, 3, 4, 0, 0, 0]
# concat(seq1, seq2) [[1, 2, 3, 4, 0, 0, 0],
#                     [9, 7, 6, 4, 3, 7, 5]]

In [7]:
def collate_fn(batch):
    data = []
    target = []
    for item in batch:
        data.append(torch.as_tensor(item['data']))
        target.append(torch.as_tensor(item['target']))
    # pad different length sequences
    data = pad_sequence(data, batch_first=True, padding_value=0)
    target = pad_sequence(target, batch_first=True, padding_value=0)

    return {'data': data, 'target': target}

In [8]:
class RNNCellPredictor(nn.Module):
    def __init__(self, vocab_size, emb_dim, hidden_dim, n_classes):
        super().__init__()
        self.word_emb = nn.Embedding(vocab_size, emb_dim)
        self.rnn_cell = nn.GRUCell(emb_dim, hidden_dim)
        self.clf = nn.Linear(hidden_dim, n_classes)
        self.do = nn.Dropout(0.1)
        self.hidden_dim = hidden_dim

    def forward(self, x): # B x T
        b, t = x.size()
        emb = self.word_emb(x) # B x T x Ebm_dim
        rnn_out = []
        hidden = torch.zeros((b, self.hidden_dim), device=x.device)
        for i in range(t):
            hidden = self.gru_cell(emb[:, i, :], # emb[:, i, :]: B x Emb_dim
                                   hidden) # hidden: B x Hid_dim
            rnn_out.append(hidden.unsqueeze(1)) # B x 1 x Hid_dim
        rnn_out = torch.cat(rnn_out, dim=1) # B x T x Hid_dim

        return self.clf(self.do(rnn_out))


In [9]:
# #hyper params
# vocab_size = len(dataset.word_vocab) + 1
# n_classes = len(dataset.target_vocab) + 1
# n_chars = len(dataset.char_vocab) + 1
# #TODO try to use other model parameters
# emb_dim = 256
# hidden = 256
# n_epochs = 10
# cuda_device = 0
# batch_size = 100
# device = f'cuda:{cuda_device}' if cuda_device != -1 else 'cpu'

In [10]:
#hyper params
vocab_size = len(dataset.word_vocab) + 1
n_classes = len(dataset.target_vocab) + 1
n_chars = len(dataset.char_vocab) + 1
#TODO try to use other model parameters
emb_dim = 128 # поменял с 256 на 128
hidden = 256
char_hid = 64
char_emb = 32
n_epochs = 10
batch_size = 64
cuda_device = 0
batch_size = 100
device = f'cuda:{cuda_device}' if cuda_device != -1 else 'cpu'

In [11]:
class RNN_GRU(nn.Module):
    def __init__(self, vocab_size, emb_dim, hidden_dim, n_classes):
        super().__init__()
        #TODO try to use other RNN archicetures, f.e. RNN and LSTM
        self.word_emb = nn.Embedding(vocab_size, emb_dim)
        # batch_first = False: T x B x Vec
        # batch_first = True: B x T x Vec
        self.rnn = nn.GRU(emb_dim, hidden_dim, batch_first=True) 
        self.clf = nn.Linear(hidden_dim, n_classes)
        self.do = nn.Dropout(0.1)
        self.hidden_dim = hidden_dim

    def forward(self, x):
        emb = self.word_emb(x)
        hidden, _ = self.rnn(emb)

        return self.clf(self.do(hidden))
        

In [12]:
model1 = RNN_GRU(vocab_size, emb_dim, hidden, n_classes).to(device)
model1.train()
optim = torch.optim.Adam(model1.parameters(), lr=0.001)
loss_func = nn.CrossEntropyLoss()

In [13]:
%%time
start1 = datetime.datetime.now()
for epoch in range(n_epochs):
    dataloader = DataLoader(dataset, 
                            batch_size, 
                            shuffle=True, 
                            collate_fn=collate_fn,
                            drop_last = True,
                            )
    for i, batch in enumerate(dataloader):
        optim.zero_grad()

        predict = model1(batch['data'].to(device))
        loss = loss_func(predict.view(-1, n_classes),
                         batch['target'].to(device).view(-1), 
                         )
        loss.backward()
        optim.step()
        if i % 100 == 0:
            print(f'epoch: {epoch}, step: {i}, loss: {loss.item()}')
   
    torch.save(model1.state_dict(), f'./rnn_chkpt_{epoch}.pth')
GRU_train_time = datetime.datetime.now() - start1
GRU_train_loss = loss.item()
print(GRU_train_time)
print(GRU_train_loss)

epoch: 0, step: 0, loss: 2.697464942932129
epoch: 0, step: 100, loss: 0.34761226177215576
epoch: 0, step: 200, loss: 0.3428894579410553
epoch: 1, step: 0, loss: 0.21351249516010284
epoch: 1, step: 100, loss: 0.15344545245170593
epoch: 1, step: 200, loss: 0.1469312161207199
epoch: 2, step: 0, loss: 0.13336654007434845
epoch: 2, step: 100, loss: 0.11462906748056412
epoch: 2, step: 200, loss: 0.09780871123075485
epoch: 3, step: 0, loss: 0.09811171144247055
epoch: 3, step: 100, loss: 0.1071963906288147
epoch: 3, step: 200, loss: 0.06952792406082153
epoch: 4, step: 0, loss: 0.06481955200433731
epoch: 4, step: 100, loss: 0.09193825721740723
epoch: 4, step: 200, loss: 0.05829273536801338
epoch: 5, step: 0, loss: 0.07658292353153229
epoch: 5, step: 100, loss: 0.06946925073862076
epoch: 5, step: 200, loss: 0.05656103044748306
epoch: 6, step: 0, loss: 0.07207252085208893
epoch: 6, step: 100, loss: 0.0842195674777031
epoch: 6, step: 200, loss: 0.05789846181869507
epoch: 7, step: 0, loss: 0.056710

In [14]:
#example
phrase = 'He ran quickly after the red bus and caught it'
words = phrase.split(' ')
tokens = [dataset.word_vocab[w] for w in words]

start11 = datetime.datetime.now()
with torch.no_grad():
    model1.eval()
    predict = model1(torch.tensor(tokens).unsqueeze(0).to(device)) # 1 x T x N_classes
    labels = torch.argmax(predict, dim=-1).squeeze().cpu().detach().tolist()
    GRU_inference_time = datetime.datetime.now() - start11

target_labels = list(dataset.target_vocab.keys())
print([target_labels[l-1] for l in labels])
print(GRU_inference_time)

['PRON', 'VERB', 'ADV', 'ADP', 'DET', 'ADJ', 'NOUN', 'CCONJ', 'VERB', 'PRON']
0:00:00.001855


In [15]:
class RNN_LSTM(nn.Module):
    def __init__(self, vocab_size, emb_dim, hidden_dim, n_classes):
        super().__init__()
        #TODO try to use other RNN archicetures, f.e. RNN and LSTM
        self.word_emb = nn.Embedding(vocab_size, emb_dim)
        # batch_first = False: T x B x Vec
        # batch_first = True: B x T x Vec
        self.rnn = nn.LSTM(emb_dim, hidden_dim, batch_first=True) 
        self.clf = nn.Linear(hidden_dim, n_classes)
        self.do = nn.Dropout(0.1)
        self.hidden_dim = hidden_dim

    def forward(self, x):
        emb = self.word_emb(x)
        hidden, _ = self.rnn(emb)

        return self.clf(self.do(hidden))

In [16]:
model2 = RNN_LSTM(vocab_size, emb_dim, hidden, n_classes).to(device)
model2.train()
optim = torch.optim.Adam(model2.parameters(), lr=0.001)
loss_func = nn.CrossEntropyLoss()

In [17]:
%%time
start2 = datetime.datetime.now()
for epoch in range(n_epochs):
    dataloader = DataLoader(dataset, 
                            batch_size, 
                            shuffle=True, 
                            collate_fn=collate_fn,
                            drop_last = True,
                            )
    for i, batch in enumerate(dataloader):
        optim.zero_grad()

        predict = model2(batch['data'].to(device))
        loss = loss_func(predict.view(-1, n_classes),
                         batch['target'].to(device).view(-1), 
                         )
        loss.backward()
        optim.step()
        if i % 100 == 0:
            print(f'epoch: {epoch}, step: {i}, loss: {loss.item()}')
   
    torch.save(model2.state_dict(), f'./rnn_chkpt_{epoch}.pth')
LSTM_train_time = datetime.datetime.now() - start2
LSTM_train_loss = loss.item()
print(LSTM_train_time)
print(LSTM_train_loss)

epoch: 0, step: 0, loss: 2.7950456142425537
epoch: 0, step: 100, loss: 0.3105144202709198
epoch: 0, step: 200, loss: 0.3319418430328369
epoch: 1, step: 0, loss: 0.23660987615585327
epoch: 1, step: 100, loss: 0.2564547061920166
epoch: 1, step: 200, loss: 0.21276871860027313
epoch: 2, step: 0, loss: 0.18634840846061707
epoch: 2, step: 100, loss: 0.1283894032239914
epoch: 2, step: 200, loss: 0.1633543074131012
epoch: 3, step: 0, loss: 0.1179714947938919
epoch: 3, step: 100, loss: 0.0923578068614006
epoch: 3, step: 200, loss: 0.07840301096439362
epoch: 4, step: 0, loss: 0.09804749488830566
epoch: 4, step: 100, loss: 0.09709502756595612
epoch: 4, step: 200, loss: 0.060109544545412064
epoch: 5, step: 0, loss: 0.05985260754823685
epoch: 5, step: 100, loss: 0.08648263663053513
epoch: 5, step: 200, loss: 0.04569769278168678
epoch: 6, step: 0, loss: 0.05195637047290802
epoch: 6, step: 100, loss: 0.08191590011119843
epoch: 6, step: 200, loss: 0.08082637935876846
epoch: 7, step: 0, loss: 0.0361664

In [18]:
#example
phrase = 'He ran quickly after the red bus and caught it'
words = phrase.split(' ')
tokens = [dataset.word_vocab[w] for w in words]

start22 = datetime.datetime.now()
with torch.no_grad():
    model2.eval()
    predict = model2(torch.tensor(tokens).unsqueeze(0).to(device)) # 1 x T x N_classes
    labels = torch.argmax(predict, dim=-1).squeeze().cpu().detach().tolist()
    LSTM_inference_time = datetime.datetime.now() - start22

target_labels = list(dataset.target_vocab.keys())
print([target_labels[l-1] for l in labels])
print(LSTM_inference_time)

['PRON', 'VERB', 'ADV', 'SCONJ', 'DET', 'ADJ', 'NOUN', 'CCONJ', 'VERB', 'PRON']
0:00:00.004043


In [19]:
class RNN(nn.Module):
    def __init__(self, vocab_size, emb_dim, hidden_dim, n_classes):
        super().__init__()
        #TODO try to use other RNN archicetures, f.e. RNN and LSTM
        self.word_emb = nn.Embedding(vocab_size, emb_dim)
        # batch_first = False: T x B x Vec
        # batch_first = True: B x T x Vec
        self.rnn = nn.RNN(emb_dim, hidden_dim, batch_first=True) 
        self.clf = nn.Linear(hidden_dim, n_classes)
        self.do = nn.Dropout(0.1)
        self.hidden_dim = hidden_dim

    def forward(self, x):
        emb = self.word_emb(x)
        hidden, _ = self.rnn(emb)

        return self.clf(self.do(hidden))

In [20]:
model3 = RNN_LSTM(vocab_size, emb_dim, hidden, n_classes).to(device)
model3.train()
optim = torch.optim.Adam(model3.parameters(), lr=0.001)
loss_func = nn.CrossEntropyLoss()

In [21]:
%%time
start3 = datetime.datetime.now()
for epoch in range(n_epochs):
    dataloader = DataLoader(dataset, 
                            batch_size, 
                            shuffle=True, 
                            collate_fn=collate_fn,
                            drop_last = True,
                            )
    for i, batch in enumerate(dataloader):
        optim.zero_grad()

        predict = model3(batch['data'].to(device))
        loss = loss_func(predict.view(-1, n_classes),
                         batch['target'].to(device).view(-1), 
                         )
        loss.backward()
        optim.step()
        if i % 100 == 0:
            print(f'epoch: {epoch}, step: {i}, loss: {loss.item()}')
   
    torch.save(model3.state_dict(), f'./rnn_chkpt_{epoch}.pth')
RNN_train_time = datetime.datetime.now() - start3
RNN_train_loss = loss.item()
print(RNN_train_time)
print(RNN_train_loss)

epoch: 0, step: 0, loss: 2.8822274208068848
epoch: 0, step: 100, loss: 0.5095211267471313
epoch: 0, step: 200, loss: 0.3305400013923645
epoch: 1, step: 0, loss: 0.21899423003196716
epoch: 1, step: 100, loss: 0.20709535479545593
epoch: 1, step: 200, loss: 0.1597582846879959
epoch: 2, step: 0, loss: 0.1753382831811905
epoch: 2, step: 100, loss: 0.14058679342269897
epoch: 2, step: 200, loss: 0.1603178232908249
epoch: 3, step: 0, loss: 0.1254953294992447
epoch: 3, step: 100, loss: 0.11292046308517456
epoch: 3, step: 200, loss: 0.10796342045068741
epoch: 4, step: 0, loss: 0.08759182691574097
epoch: 4, step: 100, loss: 0.08134536445140839
epoch: 4, step: 200, loss: 0.07035689800977707
epoch: 5, step: 0, loss: 0.06422707438468933
epoch: 5, step: 100, loss: 0.07264790683984756
epoch: 5, step: 200, loss: 0.07298268377780914
epoch: 6, step: 0, loss: 0.038619861006736755
epoch: 6, step: 100, loss: 0.039593227207660675
epoch: 6, step: 200, loss: 0.054011519998311996
epoch: 7, step: 0, loss: 0.0522

In [22]:
#example
phrase = 'He ran quickly after the red bus and caught it'
words = phrase.split(' ')
tokens = [dataset.word_vocab[w] for w in words]

start33 = datetime.datetime.now()
with torch.no_grad():
    model3.eval()
    predict = model3(torch.tensor(tokens).unsqueeze(0).to(device)) # 1 x T x N_classes
    labels = torch.argmax(predict, dim=-1).squeeze().cpu().detach().tolist()
    RNN_inference_time = datetime.datetime.now() - start33

target_labels = list(dataset.target_vocab.keys())
print([target_labels[l-1] for l in labels])
print(RNN_inference_time)

['PRON', 'VERB', 'ADV', 'SCONJ', 'DET', 'ADJ', 'NOUN', 'CCONJ', 'VERB', 'PRON']
0:00:00.001290


In [23]:
def collate_fn1(input_data):
    data = []
    chars = []
    targets = []
    max_len = 0
    for item in input_data:
        if len(item['data']) > max_len:
            max_len = len(item['data'])
        data.append(torch.as_tensor(item['data']))
        chars.append(item['char'])
        targets.append(torch.as_tensor(item['target']))
    chars_seq = [[torch.as_tensor([0]) for _ in range(len(input_data))] for _ in range(max_len)]
    for j in range(len(input_data)):
        for i in range(max_len):
            if len(chars[j]) > i:
                chars_seq[i][j] = torch.as_tensor(chars[j][i])
    for j in range(max_len):
        chars_seq[j] = pad_sequence(chars_seq[j], batch_first=True, padding_value=0)
    data = pad_sequence(data, batch_first=True, padding_value=0)
    targets = pad_sequence(targets, batch_first=True, padding_value=0)
    return {'data': data, 'chars': chars_seq, 'target': targets}

In [24]:
class CharRNN_GRU(nn.Module):
    def __init__(self, vocab_size, emb_dim, hidden_dim):
        super().__init__()
        self.char_emb = nn.Embedding(vocab_size, emb_dim)
        self.rnn = nn.GRU(emb_dim, hidden_dim, batch_first=True)

    def forward(self, x):
        emb = self.char_emb(x) # B x T x Emb_dim
        _, out = self.rnn(emb)
        # _: B x T x Hidden 
        # out: 1 x B x Hidden

        return out.transpose(0, 1) # B x 1 x Hidden

In [25]:
class RNN_GRU_CH(nn.Module):
    def __init__(self, vocab_size, emb_dim, hidden_dim, n_classes,
                 char_vocab, char_emb, char_hidden):
        super().__init__()
        #TODO try to use other RNN archicetures, f.e. RNN and LSTM
        self.word_emb = nn.Embedding(vocab_size, emb_dim)
        # batch_first = False: T x B x Vec
        # batch_first = True: B x T x Vec
        self.rnn = nn.GRU(emb_dim + char_hidden, hidden_dim, batch_first=True) 
        self.clf = nn.Linear(hidden_dim, n_classes)
        self.do = nn.Dropout(0.1)
        self.hidden_dim = hidden_dim
        self.char_rnn = CharRNN_GRU(char_vocab, char_emb, char_hidden)

    def forward(self, x, chars):
        emb = self.word_emb(x)
        char_features = [self.char_rnn(c.to(x.device)) for c in chars]
        char_features = torch.cat(char_features, dim=1) # конкатенация по времени B x T x Char_hid
        emb = torch.cat((emb, char_features), dim=-1) # конкатенация векторов
        hidden, _ = self.rnn(emb)

        return self.clf(self.do(hidden))

In [26]:
model4 = RNN_GRU_CH(vocab_size, emb_dim, hidden, n_classes, n_chars, char_emb, char_hid).to(device)
model4.train()
optim = torch.optim.Adam(model4.parameters(), lr=0.001)
loss_func = nn.CrossEntropyLoss()

In [27]:
start4 = datetime.datetime.now()
for epoch in range(n_epochs):
    dataloader = DataLoader(dataset, 
                            batch_size, 
                            shuffle=True, 
                            collate_fn=collate_fn1,
                            drop_last = True,
                            )
    for i, batch in enumerate(dataloader):
        optim.zero_grad()

        predict = model4(batch['data'].to(device), batch['chars'])
        loss = loss_func(predict.view(-1, n_classes),
                         batch['target'].to(device).view(-1), 
                         )
        loss.backward()
        optim.step()
        if i % 100 == 0:
            print(f'epoch: {epoch}, step: {i}, loss: {loss.item()}')
   
    torch.save(model4.state_dict(), f'./rnn_chkpt_{epoch}.pth')
GRUch_train_time = datetime.datetime.now() - start4
GRUch_train_loss = loss.item()
print(GRUch_train_time)
print(GRUch_train_loss)

epoch: 0, step: 0, loss: 2.6814215183258057
epoch: 0, step: 100, loss: 0.290275901556015
epoch: 0, step: 200, loss: 0.06449785083532333
epoch: 1, step: 0, loss: 0.15950611233711243
epoch: 1, step: 100, loss: 0.18168722093105316
epoch: 1, step: 200, loss: 0.15610025823116302
epoch: 2, step: 0, loss: 0.106880784034729
epoch: 2, step: 100, loss: 0.09623966366052628
epoch: 2, step: 200, loss: 0.07311837375164032
epoch: 3, step: 0, loss: 0.05631289258599281
epoch: 3, step: 100, loss: 0.09291616082191467
epoch: 3, step: 200, loss: 0.07114402204751968
epoch: 4, step: 0, loss: 0.06549698859453201
epoch: 4, step: 100, loss: 0.06335575878620148
epoch: 4, step: 200, loss: 0.07194525003433228
epoch: 5, step: 0, loss: 0.051111310720443726
epoch: 5, step: 100, loss: 0.05010803043842316
epoch: 5, step: 200, loss: 0.06252729892730713
epoch: 6, step: 0, loss: 0.05802266299724579
epoch: 6, step: 100, loss: 0.0704229399561882
epoch: 6, step: 200, loss: 0.05308810994029045
epoch: 7, step: 0, loss: 0.05361

In [28]:
#example
#TODO modify inference for model with char input
phrase = 'He ran quickly after the red bus and caught it'
words = phrase.split(' ')
tokens = [dataset.word_vocab[w] for w in words]
chars = [torch.tensor([dataset.char_vocab[c] for c in w]).unsqueeze(0).to(device) for w in words]

start44 = datetime.datetime.now()
with torch.no_grad():
    model4.eval()
    predict = model4(torch.tensor(tokens).unsqueeze(0).to(device), chars) # 1 x T x N_classes
    labels = torch.argmax(predict, dim=-1).squeeze().cpu().detach().tolist()
    GRUch_inference_time = datetime.datetime.now() - start44

target_labels = list(dataset.target_vocab.keys())
print([target_labels[l-1] for l in labels])
print(GRUch_inference_time)

['PRON', 'VERB', 'ADV', 'ADV', 'DET', 'ADJ', 'NOUN', 'CCONJ', 'VERB', 'PRON']
0:00:00.005445


In [30]:
class RNN_LSTM_CH(nn.Module):
    def __init__(self, vocab_size, emb_dim, hidden_dim, n_classes,
                 char_vocab, char_emb, char_hidden):
        super().__init__()
        #TODO try to use other RNN archicetures, f.e. RNN and LSTM
        self.word_emb = nn.Embedding(vocab_size, emb_dim)
        # batch_first = False: T x B x Vec
        # batch_first = True: B x T x Vec
        self.rnn = nn.LSTM(emb_dim + char_hidden, hidden_dim, batch_first=True) 
        self.clf = nn.Linear(hidden_dim, n_classes)
        self.do = nn.Dropout(0.1)
        self.hidden_dim = hidden_dim
        self.char_rnn = CharRNN_GRU(char_vocab, char_emb, char_hidden)
# При создании  класса CharRNN на основе LSTM при тренировке модели аозникант ошибка - attributeerror: tuple object has no attribute transpose. ПОЧЕМУ???
    def forward(self, x, chars):
        emb = self.word_emb(x)
        char_features = [self.char_rnn(c.to(x.device)) for c in chars]
        char_features = torch.cat(char_features, dim=1) # конкатенация по времени B x T x Char_hid
        emb = torch.cat((emb, char_features), dim=-1) # конкатенация векторов
        hidden, _ = self.rnn(emb)

        return self.clf(self.do(hidden))

In [31]:
model5 = RNN_LSTM_CH(vocab_size, emb_dim, hidden, n_classes, n_chars, char_emb, char_hid).to(device)
model5.train()
optim = torch.optim.Adam(model5.parameters(), lr=0.001)
loss_func = nn.CrossEntropyLoss()

In [32]:
start5 = datetime.datetime.now()
for epoch in range(n_epochs):
    dataloader = DataLoader(dataset, 
                            batch_size, 
                            shuffle=True, 
                            collate_fn=collate_fn1,
                            drop_last = True,
                            )
    for i, batch in enumerate(dataloader):
        optim.zero_grad()

        predict = model5(batch['data'].to(device), batch['chars'])
        loss = loss_func(predict.view(-1, n_classes),
                         batch['target'].to(device).view(-1), 
                         )
        loss.backward()
        optim.step()
        if i % 100 == 0:
            print(f'epoch: {epoch}, step: {i}, loss: {loss.item()}')
   
    torch.save(model5.state_dict(), f'./rnn_chkpt_{epoch}.pth')
LSTMch_train_time = datetime.datetime.now() - start5
LSTMch_train_loss = loss.item()
print(LSTMch_train_time)
print(LSTMch_train_loss)

epoch: 0, step: 0, loss: 2.7824697494506836
epoch: 0, step: 100, loss: 0.34619492292404175
epoch: 0, step: 200, loss: 0.19031865894794464
epoch: 1, step: 0, loss: 0.19573692977428436
epoch: 1, step: 100, loss: 0.20226818323135376
epoch: 1, step: 200, loss: 0.1293405294418335
epoch: 2, step: 0, loss: 0.10871847718954086
epoch: 2, step: 100, loss: 0.12867826223373413
epoch: 2, step: 200, loss: 0.07060105353593826
epoch: 3, step: 0, loss: 0.05773886665701866
epoch: 3, step: 100, loss: 0.08090445399284363
epoch: 3, step: 200, loss: 0.06863255798816681
epoch: 4, step: 0, loss: 0.08211454004049301
epoch: 4, step: 100, loss: 0.06651565432548523
epoch: 4, step: 200, loss: 0.06578901410102844
epoch: 5, step: 0, loss: 0.053666356950998306
epoch: 5, step: 100, loss: 0.0725189670920372
epoch: 5, step: 200, loss: 0.05056532844901085
epoch: 6, step: 0, loss: 0.052004821598529816
epoch: 6, step: 100, loss: 0.059583988040685654
epoch: 6, step: 200, loss: 0.058589328080415726
epoch: 7, step: 0, loss: 0

In [33]:
#example
#TODO modify inference for model with char input
phrase = 'He ran quickly after the red bus and caught it'
words = phrase.split(' ')
tokens = [dataset.word_vocab[w] for w in words]
chars = [torch.tensor([dataset.char_vocab[c] for c in w]).unsqueeze(0).to(device) for w in words]

start55 = datetime.datetime.now()
with torch.no_grad():
    model5.eval()
    predict = model5(torch.tensor(tokens).unsqueeze(0).to(device), chars) # 1 x T x N_classes
    labels = torch.argmax(predict, dim=-1).squeeze().cpu().detach().tolist()
    LSTMch_inference_time = datetime.datetime.now() - start55

target_labels = list(dataset.target_vocab.keys())
print([target_labels[l-1] for l in labels])
print(LSTMch_inference_time)

['PRON', 'VERB', 'ADV', 'ADP', 'DET', 'ADJ', 'NOUN', 'CCONJ', 'VERB', 'PRON']
0:00:00.007477


In [34]:
class RNN_CH(nn.Module):
    def __init__(self, vocab_size, emb_dim, hidden_dim, n_classes,
                 char_vocab, char_emb, char_hidden):
        super().__init__()
        #TODO try to use other RNN archicetures, f.e. RNN and LSTM
        self.word_emb = nn.Embedding(vocab_size, emb_dim)
        # batch_first = False: T x B x Vec
        # batch_first = True: B x T x Vec
        self.rnn = nn.RNN(emb_dim + char_hidden, hidden_dim, batch_first=True) 
        self.clf = nn.Linear(hidden_dim, n_classes)
        self.do = nn.Dropout(0.1)
        self.hidden_dim = hidden_dim
        self.char_rnn = CharRNN_GRU(char_vocab, char_emb, char_hidden)

    def forward(self, x, chars):
        emb = self.word_emb(x)
        char_features = [self.char_rnn(c.to(x.device)) for c in chars]
        char_features = torch.cat(char_features, dim=1) # конкатенация по времени B x T x Char_hid
        emb = torch.cat((emb, char_features), dim=-1) # конкатенация векторов
        hidden, _ = self.rnn(emb)

        return self.clf(self.do(hidden))

In [35]:
model6 = RNN_LSTM_CH(vocab_size, emb_dim, hidden, n_classes, n_chars, char_emb, char_hid).to(device)
model6.train()
optim = torch.optim.Adam(model6.parameters(), lr=0.001)
loss_func = nn.CrossEntropyLoss()

In [36]:
start6 = datetime.datetime.now()
for epoch in range(n_epochs):
    dataloader = DataLoader(dataset, 
                            batch_size, 
                            shuffle=True, 
                            collate_fn=collate_fn1,
                            drop_last = True,
                            )
    for i, batch in enumerate(dataloader):
        optim.zero_grad()

        predict = model6(batch['data'].to(device), batch['chars'])
        loss = loss_func(predict.view(-1, n_classes),
                         batch['target'].to(device).view(-1), 
                         )
        loss.backward()
        optim.step()
        if i % 100 == 0:
            print(f'epoch: {epoch}, step: {i}, loss: {loss.item()}')
   
    torch.save(model6.state_dict(), f'./rnn_chkpt_{epoch}.pth')
RNNch_train_time = datetime.datetime.now() - start6
RNNch_train_loss = loss.item()
print(RNNch_train_time)
print(RNNch_train_loss)

epoch: 0, step: 0, loss: 2.898219347000122
epoch: 0, step: 100, loss: 0.3857155740261078
epoch: 0, step: 200, loss: 0.25762447714805603
epoch: 1, step: 0, loss: 0.19653448462486267
epoch: 1, step: 100, loss: 0.12921006977558136
epoch: 1, step: 200, loss: 0.13722196221351624
epoch: 2, step: 0, loss: 0.17779207229614258
epoch: 2, step: 100, loss: 0.15846620500087738
epoch: 2, step: 200, loss: 0.050216637551784515
epoch: 3, step: 0, loss: 0.06497295200824738
epoch: 3, step: 100, loss: 0.09796146303415298
epoch: 3, step: 200, loss: 0.10486404597759247
epoch: 4, step: 0, loss: 0.07948961108922958
epoch: 4, step: 100, loss: 0.06036562845110893
epoch: 4, step: 200, loss: 0.053657907992601395
epoch: 5, step: 0, loss: 0.04653461277484894
epoch: 5, step: 100, loss: 0.063311867415905
epoch: 5, step: 200, loss: 0.06128644943237305
epoch: 6, step: 0, loss: 0.06529584527015686
epoch: 6, step: 100, loss: 0.05953168496489525
epoch: 6, step: 200, loss: 0.06656263768672943
epoch: 7, step: 0, loss: 0.046

In [37]:
#example
#TODO modify inference for model with char input
phrase = 'He ran quickly after the red bus and caught it'
words = phrase.split(' ')
tokens = [dataset.word_vocab[w] for w in words]
chars = [torch.tensor([dataset.char_vocab[c] for c in w]).unsqueeze(0).to(device) for w in words]

start66 = datetime.datetime.now()
with torch.no_grad():
    model5.eval()
    predict = model5(torch.tensor(tokens).unsqueeze(0).to(device), chars) # 1 x T x N_classes
    labels = torch.argmax(predict, dim=-1).squeeze().cpu().detach().tolist()
    RNNch_inference_time = datetime.datetime.now() - start66

target_labels = list(dataset.target_vocab.keys())
print([target_labels[l-1] for l in labels])
print(RNNch_inference_time)

['PRON', 'VERB', 'ADV', 'ADP', 'DET', 'ADJ', 'NOUN', 'CCONJ', 'VERB', 'PRON']
0:00:00.007104


In [39]:
result = pd.DataFrame(data=[[GRU_train_time, GRU_train_loss, GRU_inference_time],
                            [LSTM_train_time, LSTM_train_loss, LSTM_inference_time],
                            [RNN_train_time, RNN_train_loss, RNN_inference_time],
                            [GRUch_train_time, GRUch_train_loss, GRUch_inference_time],
                            [LSTMch_train_time, LSTMch_train_loss, LSTMch_inference_time],
                            [RNNch_train_time, RNNch_train_loss, RNNch_inference_time]],
                      index = ['GRU','LSTM','RNN', 'GRUch','LSTMch','RNNch'],
                      columns = ['Время обучения', 'Loss на обучающей выборке', 'Время инференса'])
result

,Время обучения,Loss на обучающей выборке,Время инференса
GRU,0 days 00:00:23.645993,0.050931,0 days 00:00:00.001855
LSTM,0 days 00:00:27.226089,0.036536,0 days 00:00:00.004043
RNN,0 days 00:00:25.665279,0.031575,0 days 00:00:00.001290
GRUch,0 days 00:04:42.053768,0.042818,0 days 00:00:00.005445
LSTMch,0 days 00:04:55.345859,0.028545,0 days 00:00:00.007477
RNNch,0 days 00:04:23.559576,0.045305,0 days 00:00:00.007104
